In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm

import warnings 
warnings.filterwarnings(action='ignore')

from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/SeoulHangangB.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [2]:
train = pd.read_csv('dataset/data/train.csv')

In [121]:
test = pd.read_csv('dataset/test_date.csv')

In [122]:
test.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '년', '월', '일'],
      dtype='object')

## 식사가능자

In [123]:
test['식사가능자'] = test['본사정원수'] - test['본사휴가자수'] - test['본사출장자수'] - test['현본사소속재택근무자수']

## 요일평균 식계

In [124]:
def make_dow_avg(df_test, df_prev):
    dow_avg_ln = df_prev.groupby('요일').mean()['중식계']
    dow_avg_dn = df_prev.groupby('요일').mean()['석식계']
    
    df_test['요일평균중식계'] = 0
    tmp = dow_avg_ln.values
    for i in range(len(tmp)):
        df_test['요일평균중식계'][df_test['요일'] == i] = tmp[i]
        
    df_test['요일평균석식계'] = 0
    tmp = dow_avg_dn.values
    for i in range(len(tmp)):
        df_test['요일평균석식계'][df_test['요일'] == i] = tmp[i]
    
    return df_test

## 월평균식계

In [125]:
def make_month_avg(df_test, df_prev):
    month_avg_ln = df_prev.groupby('월').mean()['중식계']
    month_avg_dn = df_prev.groupby('월').mean()['석식계']
    
    df_test['월평균중식계'] = 0
    tmp = month_avg_ln
    tmp_k = tmp.keys(); tmp_v = tmp.values
    for i in tmp_k:
        df_test['월평균중식계'][df_test['월'] == i] = tmp_v[i-1]
        
    df_test['월평균석식계'] = 0
    tmp = month_avg_dn
    tmp_k = tmp.keys(); tmp_v = tmp.values
    for i in tmp_k:
        df_test['월평균석식계'][df_test['월'] == i] = tmp_v[i-1]
        
    return df_test

## 공휴일전후

In [126]:
test['공휴일전후'] = 0
test['공휴일전후'][10] =1
test['공휴일전후'][20] = 1

# 모델

## 점심

In [11]:
from pycaret.regression import *

In [23]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '년', '월', '일',
       '식사가능자', '전주중식계', '전주석식계', '요일평균중식계', '요일평균석식계', '월평균중식계', '월평균석식계',
       '공휴일전후'],
      dtype='object')

In [36]:
X_train_ln = train[['요일', '본사시간외근무명령서승인건수', '요일평균중식계', '월평균중식계', '공휴일전후',
                '본사휴가자수', '본사출장자수', '식사가능자', '중식계']]

In [37]:
reg = setup(session_id=1,
            data=X_train_ln,
            target='중식계',
            #numeric_imputation = 'mean',
            normalize = True,
            #categorical_features=['월', '요일', '공휴일전후'],
            silent=True)

,Description,Value
0,session_id,1
1,Target,중식계
2,Original Data,"(1205, 9)"
3,Missing Values,False
4,Numeric Features,6
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(843, 12)"


In [38]:
top5 = compare_models(n_select=5, sort='MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,71.6987,9188.7388,94.9770,0.7870,0.1166,0.0865,0.0190
rf,Random Forest Regressor,74.7794,9933.5249,98.8920,0.7704,0.1226,0.0908,0.0570
et,Extra Trees Regressor,75.5542,10330.2492,100.8986,0.7621,0.1261,0.0924,0.0540
lightgbm,Light Gradient Boosting Machine,76.0612,10114.2297,100.0402,0.7650,0.1241,0.0919,0.0380
huber,Huber Regressor,76.8121,10528.1556,101.7810,0.7558,0.1290,0.0929,0.0090
xgboost,Extreme Gradient Boosting,76.9038,10300.0470,100.4662,0.7603,0.1233,0.0928,0.2210
br,Bayesian Ridge,77.3097,10345.2465,101.0065,0.7603,0.1262,0.0936,0.0050
ridge,Ridge Regression,77.3135,10342.2909,101.0076,0.7604,0.1264,0.0936,0.0080
lar,Least Angle Regression,77.3360,10352.2905,101.0677,0.7602,0.1268,0.0936,0.0050
lr,Linear Regression,77.3361,10352.1659,101.0678,0.7602,0.1268,0.0936,0.5400


In [39]:
gbr_l = tune_model(create_model('gbr', criterion='mae'), optimize='MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,88.2727,13746.7424,117.2465,0.7228,0.1524,0.1099
1,82.7647,12924.7530,113.6871,0.6256,0.1237,0.0907
2,66.3824,7994.5610,89.4123,0.8100,0.1121,0.0826
3,62.3398,7374.8927,85.8772,0.8439,0.1218,0.0856
4,59.5834,5892.5583,76.7630,0.8507,0.0885,0.0673
5,69.2123,9104.5558,95.4178,0.7986,0.1243,0.0858
6,75.4561,11448.2869,106.9967,0.7419,0.1211,0.0869
7,71.9898,7911.8279,88.9485,0.7586,0.1041,0.0832
8,70.9892,8850.8715,94.0791,0.8182,0.1081,0.0836
9,70.2892,9311.9494,96.4984,0.8278,0.1409,0.0952


In [40]:
pred_holdouts = predict_model(gbr_l)
final_model_l = finalize_model(gbr_l)
final_model_l

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,70.9156,9193.5047,95.8828,0.7873,0.1200,0.0884


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='mae', init=None,
                          learning_rate=0.05, loss='ls', max_depth=4,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.05, min_impurity_split=None,
                          min_samples_leaf=2, min_samples_split=4,
                          min_weight_fraction_leaf=0.0, n_estimators=260,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=1, subsample=0.8, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

## 저녁

In [96]:
X_train_dn = train[['요일', '본사시간외근무명령서승인건수', '공휴일전후', '요일평균석식계', '월평균석식계', '본사출장자수',
                 '식사가능자', '월', '일', '석식계']]

In [97]:
reg = setup(session_id=2,
            data=X_train_dn,
            target='석식계',
            #numeric_imputation = 'mean',
            normalize = True,
            #categorical_features=['월', '요일', '공휴일전후'],
            silent=True)

,Description,Value
0,session_id,2
1,Target,석식계
2,Original Data,"(1205, 10)"
3,Missing Values,False
4,Numeric Features,6
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(843, 24)"


In [98]:
top5 = compare_models(n_select=5, sort='MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,60.2581,7575.4456,86.6464,0.5937,0.8682,0.1331,0.0630
et,Extra Trees Regressor,61.8387,8157.4263,89.7794,0.5451,0.8934,0.1328,0.0550
gbr,Gradient Boosting Regressor,62.4500,7491.1622,86.2427,0.5931,0.9307,0.1344,0.0200
lightgbm,Light Gradient Boosting Machine,63.6084,7791.5239,87.9003,0.5776,0.8972,0.1410,0.0560
xgboost,Extreme Gradient Boosting,64.6017,7860.4843,88.4012,0.5604,0.8426,0.1436,0.2270
knn,K Neighbors Regressor,71.6155,9926.4138,99.3647,0.4636,1.0003,0.1560,0.0070
huber,Huber Regressor,72.1259,11552.9276,106.9963,0.3741,1.0768,0.1438,0.0060
par,Passive Aggressive Regressor,73.7974,12126.8151,109.4854,0.3489,1.0844,0.1496,0.0090
lasso,Lasso Regression,74.7545,10976.8838,104.5064,0.4026,1.0559,0.1525,0.0060
lr,Linear Regression,75.6987,11059.7107,104.9258,0.3925,1.0509,0.1542,0.0090


In [99]:
blended_d = blend_models(top5, optimize='MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,62.0521,10125.1279,100.6237,0.5428,1.1448,0.1437
1,61.8531,7278.2191,85.3125,0.3254,0.2000,0.1416
2,61.1631,7160.0302,84.6170,0.6366,0.9858,0.1173
3,59.8715,6268.7608,79.1755,0.6039,0.6031,0.1358
4,57.4162,6987.4238,83.5908,0.6127,0.8597,0.1259
5,53.9196,5262.1323,72.5406,0.6534,0.7390,0.1139
6,59.4558,5703.5676,75.5220,0.7506,1.0034,0.1350
7,59.4769,6094.9212,78.0700,0.6836,0.9458,0.1314
8,63.7731,7589.0372,87.1151,0.6371,0.9104,0.1506
9,59.1525,8111.9828,90.0665,0.6921,1.5458,0.1067


In [102]:
params = {'learning_rate': [0.0, 0.1, 0.09, 0.089, 0.08],
                  'booster': ['gbtree', 'gblinear', 'dart']}
xgb_ = tune_model(create_model('xgboost', gpu_id=0), optimize='MAE', custom_grid=params)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,62.1833,9369.4248,96.7958,0.5769,1.1035,0.1450
1,60.3312,7256.3042,85.1839,0.3274,0.2052,0.1370
2,62.1439,7583.8779,87.0855,0.6151,0.9347,0.1231
3,56.7220,6179.6353,78.6107,0.6096,0.6057,0.1292
4,56.5045,6577.9307,81.1044,0.6354,0.7815,0.1258
5,55.5339,5508.5537,74.2196,0.6371,0.6679,0.1189
6,56.1677,5366.3232,73.2552,0.7653,0.8908,0.1243
7,55.5395,5274.3608,72.6248,0.7262,0.8180,0.1279
8,64.0048,8289.2383,91.0453,0.6036,0.7922,0.1549
9,61.7286,8596.4502,92.7170,0.6737,1.5004,0.1160


NameError: name 'Audio' is not defined

In [107]:
pred_holdouts = predict_model(blended_d)
final_model_d = finalize_model(blended_d)
final_model_d

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,61.9571,8127.8475,90.1546,0.5818,0.9736,0.1405


VotingRegressor(estimators=[('rf',
                             RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100, n_jobs=-1,
                          

# 추론

## Test 5일마다 split

In [127]:
submission_df = pd.read_csv('submission/sub_f2.csv')

In [128]:
submission_df

,일자,중식계,석식계
0,2021-01-27,904.510674,389.667138
1,2021-01-28,904.011825,443.825444
2,2021-01-29,594.587829,276.191645
3,2021-02-01,1174.937702,588.832850
4,2021-02-02,1033.847466,548.337170
5,2021-02-03,961.142505,0.000000
6,2021-02-04,981.898490,0.000000
7,2021-02-05,728.021027,0.000000
8,2021-02-08,1280.646079,0.000000
9,2021-02-09,1030.937603,0.000000


In [129]:
test.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '년', '월', '일', '식사가능자', '공휴일전후'],
      dtype='object')

In [134]:
res = []
X_test = test[['일자', '요일', '본사시간외근무명령서승인건수', '공휴일전후', '본사출장자수',
                 '식사가능자', '월', '일']]
for i in range(len(X_test)):
    if i%5 == 0:
        res.append(X_test[i:i+5])

In [135]:
res = res[1:]
len(res)

9

In [136]:
res[0]

,일자,요일,본사시간외근무명령서승인건수,공휴일전후,본사출장자수,식사가능자,월,일
5,2021-02-03,2,5,0,199,2380.0,2,3
6,2021-02-04,1,476,0,211,2364.0,2,4
7,2021-02-05,0,0,0,252,2247.0,2,5
8,2021-02-08,3,690,0,174,2333.0,2,8
9,2021-02-09,4,542,0,183,2318.0,2,9


In [137]:
from datetime import timedelta
prev = train
for idx, part_test in enumerate(res):
    part_test = make_dow_avg(part_test, prev)
    part_test = make_month_avg(part_test, prev)
    
    #pred 생성
    #pred_ln = predict_model(final_model_l, part_test.drop(['일자', '월'], axis=1))
    pred_dn = predict_model(final_model_d, part_test.drop(['일자'], axis=1))
    
    #submission 업데이트
    #submission_df['중식계'][5*(idx+1):5*(idx+1)+5] = pred_ln['Label']
    submission_df['석식계'][5*(idx+1):5*(idx+1)+5] = pred_dn['Label']


In [138]:
submission_df

,일자,중식계,석식계
0,2021-01-27,904.510674,389.667138
1,2021-01-28,904.011825,443.825444
2,2021-01-29,594.587829,276.191645
3,2021-02-01,1174.937702,588.832850
4,2021-02-02,1033.847466,548.337170
5,2021-02-03,961.142505,513.462262
6,2021-02-04,981.898490,559.514774
7,2021-02-05,728.021027,387.973353
8,2021-02-08,1280.646079,694.018465
9,2021-02-09,1030.937603,583.745168


In [139]:
submission_df.to_csv('submission/sub_f_inal.csv', index=False)

In [ ]:

train['전주중식계'] = 0
train['전주석식계'] = 0
idx = pd.DatetimeIndex(train['일자']) + timedelta(weeks=-1)
for i in range(len(train)):
    try:
        train['전주중식계'][i] = train[train['일자'] == str(idx[i])[:10]]['중식계']
        train['전주석식계'][i] = train[train['일자'] == str(idx[i])[:10]]['석식계']
    except:
        train['전주중식계'][i] = train.iloc[i,:]['중식계']
        train['전주석식계'][i] = train.iloc[i,:]['석식계']

train[['중식계', '전주중식계', '석식계', '전주석식계', '일자']].head(10)